In [1]:
import numpy as np
import tensorflow as tf

/home/tomasz_bednarski/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]


In [3]:
# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpak7fczbc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f51c439b0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)


In [5]:
# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpak7fczbc/model.ckpt.
INFO:tensorflow:loss = 13.0, step = 1
INFO:tensorflow:global_step/sec: 1389.27
INFO:tensorflow:loss = 3.99906e-05, step = 101 (0.072 sec)
INFO:tensorflow:global_step/sec: 1833.7
INFO:tensorflow:loss = 0.0535407, step = 201 (0.054 sec)
INFO:tensorflow:global_step/sec: 1736.08
INFO:tensorflow:loss = 0.0170159, step = 301 (0.058 sec)
INFO:tensorflow:global_step/sec: 2033.27
INFO:tensorflow:loss = 0.00626356, step = 401 (0.049 sec)
INFO:tensorflow:global_step/sec: 1903.68
INFO:tensorflow:loss = 0.00082317, step = 501 (0.053 sec)
INFO:tensorflow:global_step/sec: 1839.7
INFO:tensorflow:loss = 0.00012139, step = 601 (0.054 sec)
INFO:tensorflow:global_step/sec: 1806.01
INFO:tensorflow:loss = 4.46632e-05, step = 701 (0.055 sec)
INFO:tensorflow:global_step/sec: 1951.16
INFO:tensorflow:loss = 7.5814e-06, step = 801 (0.051 sec)
INFO:tensorflow:global_step/sec: 2160.99
INFO:tensorf

In [6]:
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)


INFO:tensorflow:Starting evaluation at 2018-01-21-18:44:53
INFO:tensorflow:Restoring parameters from /tmp/tmpak7fczbc/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-01-21-18:44:53
INFO:tensorflow:Saving dict for global step 1000: average_loss = 3.80467e-07, global_step = 1000, loss = 1.52187e-06
INFO:tensorflow:Starting evaluation at 2018-01-21-18:44:53
INFO:tensorflow:Restoring parameters from /tmp/tmpak7fczbc/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-01-21-18:44:54
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.00256357, global_step = 1000, loss = 0.0102543
train metrics: {'average_loss': 3.8046721e-07, 'loss': 1.5218689e-06, 'global_step': 1000}
eval metrics: {'average_loss': 0.0025635695, 'loss': 0.010254278, 'global_step': 1000}


CUSTOM MODEL

In [7]:
# Declare list of features, we only have one real-valued feature
def model_fn(features, labels, mode):
  # Build a linear model and predict values
  W = tf.get_variable("W", [1], dtype=tf.float64)   # name, shape
  b = tf.get_variable("b", [1], dtype=tf.float64)
  y = W*features['x'] + b
  # Loss sub-graph
  loss = tf.reduce_sum(tf.square(y - labels))
  # Training sub-graph
  global_step = tf.train.get_global_step()
  optimizer = tf.train.GradientDescentOptimizer(0.01)
  train = tf.group(optimizer.minimize(loss),
                   tf.assign_add(global_step, 1))
  # EstimatorSpec connects subgraphs we built to the
  # appropriate functionality.
  return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=y,
      loss=loss,
      train_op=train)


In [8]:
estimator = tf.estimator.Estimator(model_fn=model_fn)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp_8e5igkd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f2c451e80>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
# define our data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)


In [15]:
print(x_train.shape)
print(x_train.ndim)

(4,)
1


In [16]:
# train
estimator.train(input_fn=input_fn, steps=1000)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp_8e5igkd/model.ckpt.
INFO:tensorflow:loss = 3.66861650092, step = 1
INFO:tensorflow:global_step/sec: 1852.48
INFO:tensorflow:loss = 0.343834366167, step = 101 (0.054 sec)
INFO:tensorflow:global_step/sec: 2154.28
INFO:tensorflow:loss = 0.0174650282782, step = 201 (0.047 sec)
INFO:tensorflow:global_step/sec: 2316.04
INFO:tensorflow:loss = 0.00492254992326, step = 301 (0.043 sec)
INFO:tensorflow:global_step/sec: 2338.65
INFO:tensorflow:loss = 0.000281899493742, step = 401 (0.043 sec)
INFO:tensorflow:global_step/sec: 2160.76
INFO:tensorflow:loss = 1.22716942205e-05, step = 501 (0.046 sec)
INFO:tensorflow:global_step/sec: 2170.75
INFO:tensorflow:loss = 3.16442159285e-06, step = 601 (0.046 sec)
INFO:tensorflow:global_step/sec: 2212.27
INFO:tensorflow:loss = 5.54746257452e-08, step = 701 (0.045 sec)
INFO:tensorflow:global_step/sec: 2276.65
INFO:tensorflow:loss = 2.63403354028e-08, step = 801 (0.0

In [17]:
# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)


INFO:tensorflow:Starting evaluation at 2018-01-21-19:06:07
INFO:tensorflow:Restoring parameters from /tmp/tmp_8e5igkd/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-01-21-19:06:08
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 1.26936e-10
INFO:tensorflow:Starting evaluation at 2018-01-21-19:06:08
INFO:tensorflow:Restoring parameters from /tmp/tmp_8e5igkd/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-01-21-19:06:08
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101023
train metrics: {'loss': 1.2693648e-10, 'global_step': 1000}
eval metrics: {'loss': 0.010102345, 'global_step': 1000}
